In [1]:
import sys
!{sys.executable} -m pip install xgboost

In [2]:
pip install bayesian-optimization

Note: you may need to restart the kernel to use updated packages.


In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn import svm
from sklearn.metrics import accuracy_score
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import precision_score, recall_score, accuracy_score

## loading the data.

In [4]:
data=pd.read_csv("xboost.csv")

In [5]:
data.head()

,course_id,thread_id,no_of_posts,no_of_comments,no_uni_users,no_of_anonymous_msg,staff_replied,no of msgs,avg_num_words,max_words,avg_resp_time,first_post_day,msg_rate,u_chain,index_longest_post,num_views,votes,votes_Square,index_max_votes,forum_id
0,analysenumerique-001,2,9.0,2.0,5.0,2.0,0.0,11.0,12.454545,31.0,-152871.727273,2,1.359260e+09,True,4,277,2,4,0,10
1,analysenumerique-001,2,9.0,2.0,5.0,2.0,0.0,11.0,12.454545,31.0,-152871.727273,2,1.359260e+09,True,4,277,0,0,0,10
2,analysenumerique-001,2,9.0,2.0,5.0,2.0,0.0,11.0,12.454545,31.0,-152871.727273,2,1.359260e+09,True,4,277,1,1,0,10
3,analysenumerique-001,2,9.0,2.0,5.0,2.0,0.0,11.0,12.454545,31.0,-152871.727273,2,1.359260e+09,True,4,277,0,0,0,10
4,analysenumerique-001,2,9.0,2.0,5.0,2.0,0.0,11.0,12.454545,31.0,-152871.727273,2,1.359260e+09,True,4,277,0,0,0,10


## Droping unwanted coloumns

In [6]:
final_data=data.drop('course_id', axis=1)

## dividing data set to testing and training sets

In [7]:
x=final_data.drop('forum_id', axis=1)
y=final_data.forum_id
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)
x_train.head()

,thread_id,no_of_posts,no_of_comments,no_uni_users,no_of_anonymous_msg,staff_replied,no of msgs,avg_num_words,max_words,avg_resp_time,first_post_day,msg_rate,u_chain,index_longest_post,num_views,votes,votes_Square,index_max_votes
629328,403,12.0,1.0,10.0,1.0,0.0,13.0,82.538462,315.0,-142308.230769,6,1.382776e+09,True,629333,245,1,1,629322
209453,10,99.0,101.0,105.0,1.0,0.0,200.0,34.985000,436.0,-22294.550000,0,1.378351e+09,True,209555,1263,0,0,209366
470097,99,7.0,1.0,6.0,1.0,1.0,8.0,63.125000,188.0,-51883.250000,3,1.384468e+09,True,470102,66,0,0,470102
312530,8510,6.0,2.0,4.0,4.0,0.0,8.0,52.750000,101.0,-6424.250000,6,1.371433e+09,True,312534,133,0,0,312527
710434,5,456.0,318.0,587.0,15.0,5.0,774.0,87.306202,882.0,-5228.273902,2,1.380931e+09,True,710833,5796,1,1,710425


## calling the XGboost classifier:

In [8]:
model = XGBClassifier()

## fitting the model

In [9]:
model.fit(x_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [10]:
predict = model.predict(x_test)

## checking the Model accuracy:

In [11]:
accuracy_test = accuracy_score(y_test,predict)
accuracy_test

0.8360247606805804

## confusion matrix and classification report:

In [12]:
print(confusion_matrix(y_test,predict))
print(classification_report(y_test,predict))

[[27791  4003   192    22  1676    82   958]
 [ 1465 35056    50    20  1075    41   751]
 [  357   123 11537     5   186     9   307]
 [  595   580    26  1746   433    32   318]
 [ 2364  2965    54    28 16045    77   670]
 [  635   579    40    19   343  1715   350]
 [  928  1097   120    44   614     5 29687]]
              precision    recall  f1-score   support

           2       0.81      0.80      0.81     34724
           3       0.79      0.91      0.85     38458
           4       0.96      0.92      0.94     12524
           7       0.93      0.47      0.62      3730
           8       0.79      0.72      0.75     22203
           9       0.87      0.47      0.61      3681
          10       0.90      0.91      0.91     32495

    accuracy                           0.84    147815
   macro avg       0.86      0.74      0.78    147815
weighted avg       0.84      0.84      0.83    147815



## Performing Hyper-parameter tuning on XGBoost

In [13]:
pbounds = {
    'learning_rate': (0.01, 1.0),
    'n_estimators': (100, 1000),
    'max_depth': (3,10),
    'subsample': (1.0, 1.0),  
    'colsample': (1.0, 1.0),   
    'gamma': (0, 5)}


In [14]:
def xgboost_hyper_param(learning_rate,
                        n_estimators,
                        max_depth,
                        subsample,
                        colsample,
                        gamma):
    max_depth = int(max_depth)
    n_estimators = int(n_estimators)
    clf = XGBClassifier(
        max_depth=max_depth,
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        gamma=gamma)
    return np.mean(cross_val_score(clf, x_train, y_train, cv=3, scoring='roc_auc'))

In [15]:
optimizer = BayesianOptimization(
    f=xgboost_hyper_param,
    pbounds=pbounds,
    random_state=1,
)

## fitting the model

In [17]:
tuning = xgb.XGBClassifier(optimizer).fit(x_train, y_train)

In [19]:
prediction= tuning.predict(x_test)

In [25]:
print("Precision = {}".format(precision_score(y_test, prediction, average='macro')))
print("Recall = {}".format(recall_score(y_test, prediction, average='macro')))
print("Accuracy = {}".format(accuracy_score(y_test, prediction)))

Precision = 0.864419787122434
Recall = 0.7433300447819315
Accuracy = 0.8360247606805804
